In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

from utils.model_utils import load_weights

from ssd_model import DSOD300
from ssd_data import InputGenerator
from ssd_utils import PriorUtil
from ssd_metric import confusion_matrix, plot_confusion_matrix, accuracy, evaluate_results

In [ ]:
from data_voc import GTUtility
gt_util = GTUtility('./data/VOC2007test/')

In [ ]:
from data_coco import GTUtility
gt_util = GTUtility('./data/COCO/', validation=True)
gt_util = gt_util.convert_to_voc()

In [ ]:
model = DSOD300(input_shape=(300, 300, 3), num_classes=gt_util.num_classes); confidence_threshold = 0.275
#model = DSOD300(input_shape=(600, 600, 3), num_classes=gt_util.num_classes); confidence_threshold = 0.35
load_weights(model, './checkpoints/201808261035_dsod300fl_voccoco/weights.110.h5')

prior_util = PriorUtil(model)

inputs, data = gt_util.sample_batch(min(gt_util.num_samples, 10000), 0, input_size=model.image_size)
#inputs, data = gt_util.sample_batch(min(gt_util.num_samples, 10000), 0, input_size=model.image_size, preserve_aspect_ratio=True)

#gen = InputGenerator(gt_util, prior_util, min(gt_util.num_samples, 10000), model.image_size, augmentation=True)
#inputs, data = next(gen.generate(encode=False))

preds = model.predict(inputs, batch_size=1, verbose=1)

### Grid search

In [ ]:
steps = np.arange(0.05, 1, 0.05)

fmes_grid = np.zeros((len(steps)))

for i, t in enumerate(steps):
    results = [prior_util.decode(p, t) for p in preds]
    fmes = evaluate_results(data, results, gt_util, iou_thresh=0.5, max_dets=100, return_fmeasure=True)
    fmes_grid[i] = fmes
    print('threshold %.2f f-measure %.2f' % (t, fmes))

In [ ]:
max_idx = np.argmax(fmes_grid)
print(steps[max_idx], fmes_grid[max_idx])
plt.figure(figsize=[12,6])
plt.plot(steps, fmes_grid)
plt.plot(steps[max_idx], fmes_grid[max_idx], 'or')
plt.xticks(steps)
plt.grid()
plt.xlabel('threshold')
plt.ylabel('f-measure')
plt.show()

In [ ]:
confidence_threshold = 0.275 # pascal voc 2007 test
#confidence_threshold = 0.550 # pascal voc 2012 test
#confidence_threshold = 0.125 # ms coco test

### Precision-recall curve, mean Average Precision

In [ ]:
results = [prior_util.decode(p, confidence_threshold=0.01, keep_top_k=400) for p in tqdm(preds)]
evaluate_results(data, results, gt_util, iou_thresh=0.5, max_dets=100)

### Confusion matrix of local predictions

In [ ]:
results = [prior_util.decode(p, confidence_threshold) for p in tqdm(preds)]
encoded_gt = [prior_util.encode(d) for d in tqdm(data)]

y_true_all = []
y_pred_all = []

for i in range(len(data)):
    y_true = np.argmax(encoded_gt[i][:,4:], axis=1)
    y_pred = np.argmax(preds[i][:,4:], axis=1)
    #prior_object_idx = np.where(y_true)[0] # gt prior box contains object
    prior_object_idx = np.where(y_true+y_pred)[0] # gt or prediction prior box contains object
    y_true_all.extend(y_true[prior_object_idx])
    y_pred_all.extend(y_pred[prior_object_idx])
    
#y_pred_all = [ 17 for i in y_pred_all]

cm = confusion_matrix(y_true_all, y_pred_all, gt_util.num_classes)
plot_confusion_matrix(cm, gt_util.classes, figsize=[12]*2)
accuracy(y_true_all, y_pred_all)

### Examples

In [ ]:
for i in np.random.randint(0, len(inputs), 30):
    plt.figure(figsize=[8]*2)
    gt_util.plot_input(inputs[i])
    #gt_util.plot_gt(data[i])
    prior_util.plot_results(results=results[i], classes=gt_util.classes, gt_data=data[i], 
                            confidence_threshold=confidence_threshold)
    plt.axis('off')
    plt.show()